In [1]:
from PIL import Image
import os

def reduce_tiff_size(input_path, output_path, target_size_mb=15, dpi=600):
    # Convert MB to bytes
    target_size_bytes = target_size_mb * 1024 * 1024
    
    # Open the original TIFF image
    with Image.open(input_path) as img:
        # Set initial quality and check size
        quality = 95
        img.save(output_path, dpi=(dpi, dpi), compression="tiff_lzw")
        
        # Check size and iteratively reduce quality if needed
        while os.path.getsize(output_path) > target_size_bytes and quality > 10:
            quality -= 5  # Reduce quality gradually
            img.save(output_path, dpi=(dpi, dpi), compression="tiff_lzw", quality=quality)

        # Final check if the file size is still too large
        if os.path.getsize(output_path) > target_size_bytes:
            print("Warning: Could not reach target size without significant quality loss.")
        else:
            print(f"Image successfully reduced to {os.path.getsize(output_path) / (1024 * 1024):.2f} MB.")

# Example usage
input_tiff = "Fig7a.tiff"
output_tiff = "Fig7ar.tiff"

reduce_tiff_size(input_tiff, output_tiff)


Image successfully reduced to 0.77 MB.


In [7]:
from PIL import Image, ImageChops
import os

def reduce_and_crop_tiff(input_path, output_path, target_size_mb=15, dpi=600):
    # Convert MB to bytes
    target_size_bytes = target_size_mb * 1024 * 1024

    # Open the TIFF image
    with Image.open(input_path) as img:
        # Convert to RGB (if image is not in RGB mode)
        if img.mode != "RGB":
            img = img.convert("RGB")

        # Crop redundant space (assumed to be white background)
        bg = Image.new("RGB", img.size, (255, 255, 255))
        diff = ImageChops.difference(img, bg)
        bbox = diff.getbbox()
        if bbox:
            img = img.crop(bbox)

        # Save cropped image with initial settings
        img.save(output_path, dpi=(dpi, dpi), compression="tiff_lzw")

        # Reduce file size iteratively if needed
        quality = 95
        while os.path.getsize(output_path) > target_size_bytes and quality > 10:
            quality -= 5  # Decrease quality step by step
            img.save(output_path, dpi=(dpi, dpi), compression="tiff_lzw", quality=quality)

        # Final check if file is within size limits
        final_size_mb = os.path.getsize(output_path) / (1024 * 1024)
        if final_size_mb > target_size_mb:
            print("Warning: Could not reach target size without significant quality loss.")
        else:
            print(f"Image successfully reduced to {final_size_mb:.2f} MB.")

# Example usage
input_tiff = "BNN.tiff"
output_tiff = "Fig7cr.tiff"

reduce_and_crop_tiff(input_tiff, output_tiff)


Image successfully reduced to 0.64 MB.
